In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os  

In [4]:
os.getcwd() 

'/root/data/experiments/exp_siamese'

In [5]:
import sys                                                                             # Python system library needed to load custom functions
import numpy as np                                                                     # for performing calculations on numerical arrays
import pandas as pd                                                                    # home of the DataFrame construct, _the_ most important object for Data Science
import seaborn as sns                                                                  # additional plotting library
import matplotlib.pyplot as plt                                                        # allows creation of insightful plots
import os                                                                              # for changing the directory

import sagemaker                                                                       # dedicated sagemaker library to execute training jobs
import boto3                                                                           # for interacting with S3 buckets

from sagemaker.huggingface import HuggingFace                                           # for executing the trainig jobs
from sklearn.metrics import precision_recall_fscore_support, accuracy_score             # tools to understand how our model is performing

#sys.path.append('')                                                               # Add the source directory to the PYTHONPATH. This allows to import local functions and modules.
from config import DEFAULT_BUCKET, DEFAULT_REGION  
from gdsc_utils import create_encrypted_bucket, download_and_extract_model, PROJECT_DIR # functions to create S3 buckets and to help with downloading models. Importing our root directory
from gdsc_eval import plot_confusion_matrix                                             # function for creating confusion matrix                                     # importing the bucket name that contains data for the challenge and the default region
os.chdir(PROJECT_DIR)                                                                   # changing our directory to root


import logging                                                    # module for displaying relevant information in the logs
import sys                                                        # to access to some variables used or maintained by the interpreter 
import argparse                                                   # to parse arguments from passed in the hyperparameters
import json                                                       # to open the json file with labels
from transformers import (                                        # required classes to perform the model training and implement early stopping
    ASTFeatureExtractor, 
    ASTForAudioClassification, 
    Trainer, 
    TrainingArguments, 
    EarlyStoppingCallback
)                                    
import torch                                                       # library to work with PyTorch tensors and to figure out if we have a GPU available
from datasets import load_dataset, Audio, Dataset                  # required tools to create, load and process our audio dataset
from preprocessing import preprocess_audio_arrays                  # functions to preprocess the dataset with ASTFeatureExtractor
from gdsc_eval import compute_metrics, make_predictions            # functions to create predictions and evaluate them
from typing import Optional                                        # for type hints

from collections import Counter
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
def get_feature_extractor(model_name: str, 
                          train_dataset_mean: Optional[float] = None, 
                          train_dataset_std: Optional[float] = None) -> ASTFeatureExtractor:
    """
    Retrieves a feature extractor for audio signal processing.

    Args:
        model_name (str): The name of the pre-trained model to use.
        train_dataset_mean (float, optional): The mean value of the training dataset. Defaults to None.
        train_dataset_std (float, optional): The standard deviation of the training dataset. Defaults to None.

    Returns:
        ASTFeatureExtractor: An instance of the ASTFeatureExtractor class.

    """
    if all((train_dataset_mean, train_dataset_std)):
        feature_extractor = ASTFeatureExtractor.from_pretrained(model_name, mean=train_dataset_mean, std=train_dataset_std, max_length=1024)
        logger.info(f" feature extractor loaded with dataset mean: {train_dataset_mean} and standard deviation: {train_dataset_std}")
    else:
        feature_extractor = ASTFeatureExtractor.from_pretrained(model_name)
        logger.info(" at least one of the optional arguments (mean, std) is missing")
        logger.info(f" feature extractor loaded with default dataset mean: {feature_extractor.mean} and standard deviation: {feature_extractor.std}")
        
    return feature_extractor

def preprocess_data_for_training(
    dataset_path: str,
    sampling_rate: int,
    feature_extractor: ASTFeatureExtractor,
    fe_batch_size: int,
    dataset_name: str,
    shuffle: bool = False,
    extract_file_name: bool = True) -> Dataset:
    """
    Preprocesses audio data for training.

    Args:
        dataset_path (str): The path to the dataset.
        sampling_rate (int): The desired sampling rate for the audio.
        feature_extractor (ASTFeatureExtractor): The feature extractor to use for preprocessing.
        fe_batch_size (int): The batch size for feature extraction.
        dataset_name (str, optional): The name of the dataset. Defaults to None.
        shuffle (bool, optional): Whether to shuffle the dataset. Defaults to False.
        extract_file_name (bool, optional): Whether to extract paths from audio features. Defaults to True.

    Returns:
        dataset: The preprocessed dataset.

    """
    dataset = load_dataset("audiofolder", data_dir=dataset_path).get('train') # loading the dataset
    
    # perform shuffle if specified
    if shuffle:
        dataset = dataset.shuffle(seed=42)
        
    logger.info(f" loaded {dataset_name} dataset length is: {len(dataset)}")

    if extract_file_name:
        remove_metadata = lambda x: x.endswith(".wav")
        extract_file_name = lambda x: x.split('/')[-1]

        dataset_paths = list(dataset.info.download_checksums.keys())
        dataset_paths = list(filter(remove_metadata, dataset_paths))
        dataset_paths = list(map(extract_file_name, dataset_paths))
        dataset = dataset.add_column("file_name", dataset_paths)

    dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))
    
    logger.info(f" {dataset_name} dataset sampling rate casted to: {sampling_rate}")

    dataset_encoded = dataset.map(
        lambda x: preprocess_audio_arrays(x, 'audio', 'array', feature_extractor),
        remove_columns="audio",
        batched=True,
        batch_size=fe_batch_size
    )
    
    logger.info(f" done extracting features for {dataset_name} dataset")
    
    return dataset_encoded

In [8]:
parser = argparse.ArgumentParser()

# hyperparameters sent from our jupyter notebook are passed as command-line arguments to the script
# preprocessing hyperparameters
parser.add_argument("--sampling_rate", type=int, default=22050)                        # sampling rate to which we will cast audio files
parser.add_argument("--fe_batch_size", type=int, default=32)                           # feature extractor batch size
parser.add_argument("--train_dataset_mean", type=float, default=-8.076275929131292)                  # mean value of spectrograms of our data
parser.add_argument("--train_dataset_std", type=float, default=3.984092920341275)                   # standard deviation value of spectrograms of our resampled data
# parser.add_argument("--train_dataset_mean", type=float, default=-8.141991150530815)                  # mean value of spectrograms of our data
# parser.add_argument("--train_dataset_std", type=float, default=4.095692486358449)                   # standard deviation value of spectrograms of our resampled data

# training hyperparameters
parser.add_argument("--model_name", type=str)                                          # name of the pretrained model from HuggingFace
parser.add_argument("--learning_rate", type=float, default=2e-5)                       # learning rate
parser.add_argument("--epochs", type=int, default=4)                                   # number of training epochs 
parser.add_argument("--train_batch_size", type=int, default=4)                        # training batch size
parser.add_argument("--eval_batch_size", type=int, default=64)                         # evaluation batch size
parser.add_argument("--patience", type=int, default=2)                                 # early stopping - how many epoch without improvement will stop the training 
# parser.add_argument("--data_channel", type=str, default=os.environ["SM_CHANNEL_DATA"]) # directory where input data from S3 is stored
parser.add_argument("--train_dir", type=str, default="train")                          # folder name with training data
parser.add_argument("--val_dir", type=str, default="val")                              # folder name with validation data
parser.add_argument("--test_dir", type=str, default="test")                            # folder name with test data
# parser.add_argument("--output_dir", type=str, default=os.environ['SM_MODEL_DIR'])      # output directory. This directory will be saved in the S3 bucket


args, _ = parser.parse_known_args()                    # parsing arguments from the notebook


# train_path = f"{args.data_channel}/{args.train_dir}"   # directory of our training dataset on the instance
# val_path = f"{args.data_channel}/{args.val_dir}"       # directory of our validation dataset on the instance
# test_path = f"{args.data_channel}/{args.test_dir}"     # directory of our test dataset on the instance

# Set up logging which allows to print information in logs
logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.getLevelName("INFO"),
    handlers=[logging.StreamHandler(sys.stdout)],
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

In [ ]:
train_path = 'data/data_small/train'
val_path = 'data/data_small/val'

In [9]:
with open(f'data/data_small/labels.json', 'r') as f:
        labels = json.load(f)
    
# Create mapping from label to id and id to label
label2id, id2label = dict(), dict()
for k, v in labels.items():
    label2id[k] = str(v)
    id2label[str(v)] = k

num_labels = len(label2id)  # define number of labels
output_dir='models/AST',                # directory for saving model checkpoints and logs
args.model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"

In [12]:
feature_extractor = get_feature_extractor(args.model_name, args.train_dataset_mean, args.train_dataset_std)

# creating train and validation datasets
train_dataset_encoded = preprocess_data_for_training(dataset_path=train_path, sampling_rate=args.sampling_rate, feature_extractor=feature_extractor,
                                                     fe_batch_size=args.fe_batch_size, dataset_name="train", shuffle=True, extract_file_name=False)

val_dataset_encoded = preprocess_data_for_training(dataset_path=val_path, sampling_rate=args.sampling_rate, feature_extractor=feature_extractor,
                                                   fe_batch_size=args.fe_batch_size, dataset_name="validation")

# Download model from model hub
model = ASTForAudioClassification.from_pretrained(args.model_name, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)


2023-06-30 11:04:31,541 - __main__ - INFO -  feature extractor loaded with dataset mean: -8.076275929131292 and standard deviation: 3.984092920341275


Resolving data files:   0%|          | 0/177 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/default-4ea904e8a4f39112/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


...

  0%|          | 0/1 [00:00<?, ?it/s]

2023-06-30 11:04:49,909 - __main__ - INFO -  loaded validation dataset length is: 66
2023-06-30 11:04:49,915 - __main__ - INFO -  validation dataset sampling rate casted to: 22050


Map:   0%|          | 0/66 [00:00<?, ? examples/s]

2023-06-30 11:04:55,296 - __main__ - INFO -  done extracting features for validation dataset


Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([66, 768]) in the model instantiated
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([66]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
class Siamese_Dataset(torch.utils.data.Dataset):
    def __init__(self):
        self.rnd = np.random.RandomState(0)

        self.x_data = torch.tensor(train_dataset_encoded[:]['input_values'], dtype=torch.float32).to(device)
        self.y_data = torch.tensor(train_dataset_encoded[:]['label'], dtype=torch.int64).to(device) 

        self.n = len(self.x_data)
        self.singles = [k for k, v in Counter(self.y_data.tolist()).items() if v==1]

    def __len__(self):
        return self.n

#TODO: create better triplet selection
    def __getitem__(self, idx1):
        y = self.y_data[idx1]
        idx2 = self.rnd.randint(0,self.n-1)
        if idx1 == idx2:
            idx2 += 1
        y2 = self.y_data[idx2]
        idx3 = self.rnd.randint(0,self.n-1)

        if y != y2:  # get two images with same label
            while self.y_data[idx3] != y:
                idx3 += 1
                if idx3 == idx1 and not self.y_data[idx3] in self.singles:
                    idx3 += 1
                if idx3 == self.n: idx3 = 0
            idx2, idx3 = idx3, idx2
        else:  # get images different labels
            while self.y_data[idx3] == y:
                idx3 += 1
                if idx3 == self.n: idx3 = 0

        pixels1 = self.x_data[idx1]
        label1 = self.y_data[idx1] 
        pixels2 = self.x_data[idx2]
        label2 = self.y_data[idx2] 
        pixels3 = self.x_data[idx3]
        label3 = self.y_data[idx3]
        return (pixels1, label1, pixels2, label2, pixels3, label3)
    
    
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.rnd = np.random.RandomState(0)

        self.x_data = torch.tensor(train_dataset_encoded[:]['input_values'], dtype=torch.float32).to(device)
        self.y_data = torch.tensor(train_dataset_encoded[:]['label'], dtype=torch.int64).to(device) 

        self.n = len(self.x_data)

    def __len__(self):
        return self.n

    def __getitem__(self, index):
        x = self.x_data[index]
        y = self.y_data[index]

        return (x, y)

In [15]:
class SiameseNet(torch.nn.Module):
    def __init__(self):
        super(SiameseNet, self).__init__()  # pre 3.3 syntax

        self.model = ASTForAudioClassification.from_pretrained(args.model_name, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)
        self.model = self.model.audio_spectrogram_transformer
        #self.model = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
# TODO: check if we can just use AST model not ASTForAudioClassification

    def forward(self, x1, x2, x3):
        output = self.model(torch.cat((x1, x2, x3), 0)).pooler_output
        return torch.tensor_split(output, 3)
    
    
class ClassificationNet(torch.nn.Module):
    def __init__(self, siamese_net):
        super(ClassificationNet, self).__init__()  # pre 3.3 syntax
        self.model = ASTForAudioClassification.from_pretrained(args.model_name, num_labels=num_labels, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)
        self.model = self.model.classifier
        self.siamese_net = siamese_net
# TODO: check other classification architecture

    def forward(self, x):
        x = self.siamese_net(x).pooler_output
        output = self.model(x)
        return output

### TRAIN SIAMESE PART

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

batch_size = 2
max_epochs = 10
ep_log_interval = 1
lrn_rate = 2e-5


print("\nBegin MNIST Siamese network demo ")
np.random.seed(1)
torch.manual_seed(1)
train_ds = Siamese_Dataset()
train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
#TODO: check what margin would be best

# 2. create network
print("\nCreating Siamese network ")
net = SiameseNet().to(device)

# 3. train model

triplet_loss = torch.nn.TripletMarginLoss(margin=10.0, p=2)
optimizer = torch.optim.Adam(net.parameters(), lr=lrn_rate)

print("\nStarting training")
net.train()  # set mode
for epoch in range(0, max_epochs):
    ep_loss = 0  # for one full epoch
    for (batch_idx, batch) in tqdm(enumerate(train_ldr)):
        X1, y1, X2, y2, X3, y3 = batch
        oupt1, oupt2, oupt3 = net(X1, X2, X3)

        optimizer.zero_grad()       # reset gradients
        loss_val = triplet_loss(oupt1, oupt2, oupt3)

        ep_loss += loss_val.item()  # accumulate loss
        loss_val.backward()         # compute grads
        optimizer.step()            # update weights
    if epoch % ep_log_interval == 0:
        print("epoch = %4d  |  loss = %10.4f" % (epoch, ep_loss))
    print("Done ") 
print("\nEnd MNIST Siamese demo ")


##### ONLY FOR SAVING/LOADING MODEL

In [17]:
# net = SiameseNet().to(device)
# import pickle

# pickle.dump(net.model, open('model_siamese.pkl', 'wb'))
# net = torch.load('model_siamese.pkl', map_location=torch.device('cpu'))
# net.model = pickle.load(open('model_siamese.pkl', 'rb'))

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([66, 768]) in the model instantiated
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([66]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### TRAIN CLASSIFICATION PART

In [20]:
lrn_rate = 2e-5
max_epochs = 10
ep_log_interval = 1
batch_size_class = 4

train_class_ds = ClassificationDataset()
train_class_ldr = torch.utils.data.DataLoader(train_class_ds, batch_size=batch_size_class, shuffle=True)

classification_net = ClassificationNet(net.model).to(device) #pass net.model - siamese model
optimizer_class = torch.optim.Adam(classification_net.parameters(), lr=lrn_rate)
categorical_crossentropy = torch.nn.functional.cross_entropy

for epoch in range(0, max_epochs):
    ep_loss = 0  # for one full epoch
    for (batch_idx, batch) in tqdm(enumerate(train_class_ldr)):
        X, y = batch
        output = classification_net(X)

        optimizer_class.zero_grad()       # reset gradients
        loss_val = categorical_crossentropy(output, y)

        ep_loss += loss_val.item()  # accumulate loss
        loss_val.backward()         # compute grads
        optimizer_class.step()            # update weights
    if epoch % ep_log_interval == 0:
        print("epoch = %4d  |  loss = %10.4f" % (epoch, ep_loss))
    print("Done ") 

Some weights of ASTForAudioClassification were not initialized from the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 and are newly initialized because the shapes did not match:
- classifier.dense.weight: found shape torch.Size([527, 768]) in the checkpoint and torch.Size([66, 768]) in the model instantiated
- classifier.dense.bias: found shape torch.Size([527]) in the checkpoint and torch.Size([66]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
44it [00:47,  1.08s/it]


epoch =    0  |  loss =   163.8971
Done 


44it [00:48,  1.10s/it]


epoch =    1  |  loss =    74.2935
Done 


44it [00:50,  1.16s/it]


...

epoch =    7  |  loss =     0.9449
Done 


44it [00:53,  1.22s/it]


epoch =    8  |  loss =     0.7021
Done 


44it [00:53,  1.22s/it]

epoch =    9  |  loss =     0.5776
Done 


In [21]:
def make_predictions2(examples: torch.Tensor, 
                     model: torch.nn.Module, 
                     device,
                     labels: torch.Tensor = None):
    model = model.to(device)
    examples = torch.tensor(examples, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.int64).to(device) 

    with torch.no_grad():
        logits = model(examples)
    predicted_class_id = [str(torch.argmax(item).item()) for item in logits]
    if isinstance(labels, torch.Tensor):
        loss = torch.nn.functional.cross_entropy(logits.view(-1, 66), labels.to(device).view(-1), reduction="none")
        loss = loss.view(len(examples), -1).cpu().numpy()

        return {'predicted_class_id': predicted_class_id, 'loss': loss, 'logits': logits}
    else:
        return {'predicted_class_id': predicted_class_id}

    
def compute_metrics(pred, labels):
    # labels = pred.label_ids
    # preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average="macro")
    acc = accuracy_score(labels, pred)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [22]:
val_dataset_encoded2 = val_dataset_encoded.map(lambda x: make_predictions2(x['input_values'], classification_net, device, x['label']), batched=True, batch_size=4, remove_columns="input_values")

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

In [26]:
compute_metrics([int(i) for i in val_dataset_encoded2[:]['predicted_class_id']], val_dataset_encoded2[:]['label'])

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.5909090909090909,
 'f1': 0.4952380952380952,
 'precision': 0.4532828282828283,
 'recall': 0.5909090909090909}